Installation

In [ ]:
!pip install -q langchain langchain_groq langchain_community langgraph rizaio streamlit faiss-cpu sentence-transformers langsmith pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 

 Secure API Key Setup

In [ ]:
%%writefile config.py
import os

# Ensure the keys are already set; if not, raise an error.
if "GROQ_API_KEY" not in os.environ:
    raise ValueError("GROQ_API_KEY is not set!")
if "TAVILY_API_KEY" not in os.environ:
    raise ValueError("TAVILY_API_KEY is not set!")
if "RIZA_API_KEY" not in os.environ:
    raise ValueError("RIZA_API_KEY is not set!")

print("API keys loaded from environment.")


Writing config.py


Imports

In [ ]:
%%writefile imports.py
import os
import streamlit as st
from typing import Literal, List
from pydantic import BaseModel, Field
from pprint import pprint

# LangChain modules and document loaders
from langchain_core.messages import HumanMessage
from langchain.document_loaders import WikipediaLoader, ArxivLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Community tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.riza.command import ExecPython

# LangGraph modules
from langgraph.types import Command
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent

# For displaying images in notebooks
from IPython.display import Image, display

# LangSmith for logging/monitoring (if available)
try:
    import langsmith
except ImportError:
    print("LangSmith not installed.")


Writing imports.py


LLM and Tools Setup

In [ ]:
%%writefile llm_setup.py
from config import *
from imports import *
from langchain_groq import ChatGroq  # Import the ChatGroq class

# Initialize ChatGroq LLM using the API key and the chosen model.
llm = ChatGroq(groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.3-70b-versatile")

# Initialize community tools:
tool_tavily = TavilySearchResults(max_results=2)
tool_code_interpreter = ExecPython()

# Combine tools into a list (if needed later).
tools = [tool_tavily, tool_code_interpreter]

print("LLM and tools initialized.")


Writing llm_setup.py


Multi-Shot Learning Functions

In [ ]:
%%writefile multi_shot_learning.py
from imports import *
from llm_setup import llm  # Ensure llm is imported

def zero_shot_learning(query: str) -> str:
    """
    Zero-shot: Provide only the query to the LLM.
    """
    prompt = f"Answer the following question as accurately as possible:\n\n{query}"
    response = llm.invoke(prompt)
    return response.content

def one_shot_learning(query: str) -> str:
    """
    One-shot: Provide one example along with the query.
    """
    example_query = "What is the capital of France?"
    example_answer = "The capital of France is Paris."
    prompt = (
        f"Example:\nQuestion: {example_query}\nAnswer: {example_answer}\n\n"
        f"Now answer the following question:\nQuestion: {query}\nAnswer:"
    )
    response = llm.invoke(prompt)
    return response.content

def few_shot_learning(query: str) -> str:
    """
    Few-shot: Provide multiple examples along with the query.
    """
    examples = (
        "Example 1:\nQuestion: Who wrote '1984'?\nAnswer: George Orwell wrote '1984'.\n\n"
        "Example 2:\nQuestion: What is the boiling point of water?\nAnswer: The boiling point of water is 100°C at sea level.\n\n"
    )
    prompt = (
        f"{examples}"
        f"Now answer the following question in a similar style:\nQuestion: {query}\nAnswer:"
    )
    response = llm.invoke(prompt)
    return response.content

print("Multi-shot learning functions defined.")


Writing multi_shot_learning.py


RAG Integration

In [ ]:
%%writefile rag_integration.py
from imports import *

# Use the new (or fallback) document loaders:
try:
    from langchain_community.document_loaders import WikipediaLoader, ArxivLoader
except ImportError:
    from langchain.document_loaders import WikipediaLoader, ArxivLoader

def load_wikipedia_documents(query: str):
    """
    Load documents from Wikipedia based on the given query.
    """
    loader = WikipediaLoader(query)
    docs = loader.load()
    return docs

def load_arxiv_documents(query: str):
    """
    Load documents from arXiv based on the given query.
    """
    loader = ArxivLoader(query=query, max_results=3)
    docs = loader.load()
    return docs

def trim_document(doc, max_chars: int = 1000):
    """
    Trim the document's content to a maximum number of characters.
    """
    # Import Document from langchain.schema to create a new document.
    try:
        from langchain_community.schema import Document
    except ImportError:
        from langchain.schema import Document
    trimmed_content = doc.page_content[:max_chars]
    return Document(page_content=trimmed_content, metadata=doc.metadata)

def setup_vector_store_from_web(query: str, max_chars_per_doc: int = 1000):
    """
    Create a FAISS vector store by retrieving documents from Wikipedia and arXiv,
    and trimming each document's content to a maximum number of characters.
    """
    docs_wiki = load_wikipedia_documents(query)
    docs_arxiv = load_arxiv_documents(query)
    docs = docs_wiki + docs_arxiv

    # Trim each document to the specified maximum number of characters.
    trimmed_docs = [trim_document(doc, max_chars=max_chars_per_doc) for doc in docs]

    # Use updated embeddings and vectorstore if available:
    try:
        from langchain_community.embeddings import HuggingFaceEmbeddings
    except ImportError:
        from langchain.embeddings import HuggingFaceEmbeddings
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    try:
        from langchain_community.vectorstores import FAISS
    except ImportError:
        from langchain.vectorstores import FAISS
    vector_store = FAISS.from_documents(trimmed_docs, embeddings)
    return vector_store

def create_rag_chain_from_web(query: str, max_chars_per_doc: int = 1000, k: int = 2):
    """
    Create a RetrievalQA chain using a FAISS vector store built from web sources.
    """
    vector_store = setup_vector_store_from_web(query, max_chars_per_doc=max_chars_per_doc)
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,  # Make sure llm is imported from llm_setup.py below.
        chain_type="stuff",  # This is a simple chain type for demonstration.
        retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": k})
    )
    return rag_chain

def answer_with_rag_from_web(query: str, max_chars_per_doc: int = 1000, k: int = 2) -> str:
    """
    Run the RAG chain with the provided query and return the generated answer.
    """
    rag_chain = create_rag_chain_from_web(query, max_chars_per_doc=max_chars_per_doc, k=k)
    # Use invoke (the updated method) instead of run.
    result = rag_chain.invoke(query)
    return result

# Ensure that llm is imported from our LLM setup module.
from llm_setup import llm

print("RAG pipeline with Wikipedia and arXiv integration (with document trimming) set up.")


Overwriting rag_integration.py


Supervisor Node

In [ ]:
%%writefile nodes_supervisor.py
from imports import *
from llm_setup import llm  # Import llm from llm_setup.py
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
from langgraph.types import Command

class Supervisor(BaseModel):
    next: str  # Expected values: "enhancer", "researcher", or "coder"
    reason: str

system_prompt_supervisor = (
    "You are a workflow supervisor managing a team of agents: Prompt Enhancer, Researcher, and Coder. "
    "Analyze the conversation history and decide which agent should handle the next step. "
    "Choose 'enhancer' for ambiguous queries, 'researcher' for gathering information, or 'coder' for technical tasks."
)

def supervisor_node(state):
    messages = [{"role": "system", "content": system_prompt_supervisor}] + state["messages"]
    response = llm.with_structured_output(Supervisor).invoke(messages)
    goto = response.next
    reason = response.reason
    print(f"Supervisor routing to: {goto} because: {reason}")
    return Command(
        update={"messages": [HumanMessage(content=reason, name="supervisor")]},
        goto=goto
    )


Overwriting nodes_supervisor.py


Enhancer Node

In [ ]:
%%writefile nodes_enhancer.py
from imports import *
from llm_setup import llm  # Import llm
from langchain_core.messages import HumanMessage
from langgraph.types import Command

def enhancer_node(state):
    system_prompt_enhancer = (
        "You are an advanced query enhancer. Your task is to refine and clarify the user's input by removing ambiguities. "
        "Generate a more precise version of the query without asking further questions."
    )
    messages = [{"role": "system", "content": system_prompt_enhancer}] + state["messages"]
    enhanced_query = llm.invoke(messages)
    print("Enhancer: Query refined.")
    return Command(
        update={"messages": [HumanMessage(content=enhanced_query.content, name="enhancer")]},
        goto="supervisor"
    )


Overwriting nodes_enhancer.py


Researcher Node

In [ ]:
%%writefile nodes_researcher.py
from imports import *
from llm_setup import llm, tool_tavily  # Import both llm and tool_tavily
from langchain_core.messages import HumanMessage
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent

def research_node(state):
    research_agent = create_react_agent(
        llm,
        tools=[tool_tavily],
        state_modifier="You are a researcher. Focus solely on gathering and summarizing relevant information."
    )
    result = research_agent.invoke(state)
    print("Researcher: Research completed and summarized.")
    return Command(
        update={"messages": [HumanMessage(content=result['messages'][-1].content, name="researcher")]},
        goto="validator"
    )


Overwriting nodes_researcher.py


Coder Node

In [ ]:
%%writefile nodes_coder.py
from imports import *
from llm_setup import llm, tool_code_interpreter  # Import llm and tool_code_interpreter
from langchain_core.messages import HumanMessage
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent

def code_node(state):
    code_agent = create_react_agent(
        llm,
        tools=[tool_code_interpreter],
        state_modifier="You are a coder and analyst. Focus on technical problem-solving, performing calculations, and executing code as needed."
    )
    result = code_agent.invoke(state)
    print("Coder: Code execution and technical analysis complete.")
    return Command(
        update={"messages": [HumanMessage(content=result['messages'][-1].content, name="coder")]},
        goto="validator"
    )


Overwriting nodes_coder.py


Validator Node

In [ ]:
%%writefile nodes_validator.py
from imports import *
from llm_setup import llm  # Import llm
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
from langgraph.types import Command

class Validator(BaseModel):
    next: str  # Expected values: "supervisor" or "FINISH"
    reason: str

system_prompt_validator = (
    "You are a workflow validator. Review the initial user question and the final response. "
    "If the response fully answers the question, respond with 'FINISH'; otherwise, respond with 'supervisor' for further improvement."
)

def validator_node(state):
    user_question = state["messages"][0].content
    agent_answer = state["messages"][-1].content
    messages = [
        {"role": "system", "content": system_prompt_validator},
        {"role": "user", "content": user_question},
        {"role": "assistant", "content": agent_answer},
    ]
    response = llm.with_structured_output(Validator).invoke(messages)
    goto = response.next if response.next != "FINISH" else "__end__"
    decision = "Finish" if goto == "__end__" else "Continue via Supervisor"
    print(f"Validator: Decision: {decision} – Reason: {response.reason}")
    return Command(
        update={"messages": [HumanMessage(content=response.reason, name="validator")]},
        goto=goto
    )


Overwriting nodes_validator.py


State Graph Setup

In [ ]:
%%writefile state_graph_setup.py
from imports import *
from nodes_supervisor import supervisor_node
from nodes_enhancer import enhancer_node
from nodes_researcher import research_node
from nodes_coder import code_node
from nodes_validator import validator_node
from langgraph.graph import StateGraph, START, MessagesState

builder = StateGraph(MessagesState)
builder.add_node("supervisor", supervisor_node)
builder.add_node("enhancer", enhancer_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
builder.add_node("validator", validator_node)

builder.add_edge(START, "supervisor")
graph = builder.compile()

print("State Graph compiled and ready.")


Overwriting state_graph_setup.py


Graph Visualization

In [ ]:
%%writefile graph_visualization.py
from imports import *
from state_graph_setup import graph
from IPython.display import Image, display

def display_graph_diagram():
    """
    Generate and display the graph diagram as a PNG image.
    """
    graph_diagram = graph.get_graph(xray=True).draw_mermaid_png()
    display(Image(graph_diagram))
    print("Graph diagram displayed.")

if __name__ == "__main__":
    display_graph_diagram()


Overwriting graph_visualization.py


Chatbot Interface

In [ ]:
%%writefile chatbot_app.py
from imports import *
from state_graph_setup import graph
from multi_shot_learning import zero_shot_learning, one_shot_learning, few_shot_learning
from rag_integration import answer_with_rag_from_web

# Initialize session state for conversation history.
if "messages" not in st.session_state:
    st.session_state["messages"] = []  # List of tuples: (role, message)
if "learning_mode" not in st.session_state:
    st.session_state["learning_mode"] = "multi_agent"

st.title("🤖 Professional Multi-Agent Chatbot")
st.markdown("""
Welcome to the Professional Multi-Agent Chatbot integrating LangChain, LangGraph, and LangSmith.
This chatbot supports:
- **Multi-Agent Workflow:** Supervisor, Enhancer, Researcher, Coder, Validator.
- **Multi-shot Learning:** Zero-shot, One-shot, Few-shot.
- **RAG:** Retrieval Augmented Generation using Wikipedia and arXiv.
""")

learning_mode = st.sidebar.radio(
    "Select Learning Mode:",
    ("multi_agent", "zero_shot", "one_shot", "few_shot", "RAG")
)
st.session_state["learning_mode"] = learning_mode

st.markdown("### Conversation")
for role, text in st.session_state["messages"]:
    st.markdown(f"**{role.capitalize()}:** {text}")

user_input = st.text_input("Enter your message:", key="user_input")

if st.button("Send") and user_input:
    st.session_state["messages"].append(("user", user_input))
    mode = st.session_state["learning_mode"]
    if mode == "multi_agent":
        inputs = {"messages": st.session_state["messages"]}
        for output in graph.stream(inputs):
            if "messages" in output:
                for msg in output["messages"]:
                    st.session_state["messages"].append((msg.name, msg.content))
    elif mode == "zero_shot":
        answer = zero_shot_learning(user_input)
        st.session_state["messages"].append(("Zero-Shot", answer))
    elif mode == "one_shot":
        answer = one_shot_learning(user_input)
        st.session_state["messages"].append(("One-Shot", answer))
    elif mode == "few_shot":
        answer = few_shot_learning(user_input)
        st.session_state["messages"].append(("Few-Shot", answer))
    elif mode == "RAG":
        answer = answer_with_rag_from_web(user_input)
        st.session_state["messages"].append(("RAG", answer))
    # Try to rerun the app (if supported) to update the UI.
    try:
        st.experimental_rerun()
    except AttributeError:
        pass

if st.button("Clear Conversation"):
    st.session_state["messages"] = []
    try:
        st.experimental_rerun()
    except AttributeError:
        pass


Overwriting chatbot_app.py


LangSmith Logging Module

In [ ]:
%%writefile logging_module.py
from imports import *
try:
    from langsmith import Client
except ImportError:
    Client = None
    print("LangSmith not available. Skipping logging integration.")

def log_event(event_name: str, details: dict):
    """
    Log an event using LangSmith.
    """
    if Client:
        client = Client()
        client.log_event(event_name, details)
    else:
        print(f"Log: {event_name} - {details}")

print("Logging module set up.")


Overwriting logging_module.py


Utility Functions

In [ ]:
%%writefile utils.py
def format_message(role: str, message: str) -> str:
    """
    Format a message for display.
    """
    return f"**{role.capitalize()}**: {message}"

def debug_print(message: str):
    """
    Utility function for debug printing.
    """
    print(f"[DEBUG] {message}")

print("Utility functions loaded.")


Overwriting utils.py


Test Multi-Shot Learning

In [ ]:
%%writefile test_multi_shot.py
from multi_shot_learning import zero_shot_learning, one_shot_learning, few_shot_learning

if __name__ == "__main__":
    query = "What is the largest planet in our solar system?"
    print("Zero-shot:", zero_shot_learning(query))
    print("One-shot:", one_shot_learning(query))
    print("Few-shot:", few_shot_learning(query))


Overwriting test_multi_shot.py


Test RAG Integration

In [ ]:
pip install wikipedia arxiv pymupdf

In [ ]:
%%writefile test_rag.py
from rag_integration import answer_with_rag_from_web

if __name__ == "__main__":
    query = "Impact of climate change on agriculture"
    answer = answer_with_rag_from_web(query)
    print("RAG Answer:", answer)


Overwriting test_rag.py


Project Documentation

In [ ]:
%%bash
# First, kill any existing ngrok tunnels
python -c "from pyngrok import ngrok; ngrok.kill()"

# Then, launch the Streamlit app in the background using nohup.
nohup streamlit run chatbot_app.py > streamlit.log 2>&1 &
sleep 5


In [ ]:
!ngrok tunnels list

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your account at https://dash

In [ ]:
from pyngrok import ngrok
import time

# Kill any existing tunnels before attempting to create a new one.
ngrok.kill()

# Set your ngrok authtoken (optional if already set via a bash command)
ngrok.set_auth_token("2spyR1fXNbqSRjhlYoZ8o3ZNViP_5Y8JgUdHRGNRUAJghf1n9")

# Open a new tunnel on port 8501.
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://5345-34-91-10-8.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!python graph_visualization.py


/content/imports.py:9: LangChainDeprecationWarning: Importing WikipediaLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import WikipediaLoader

with new imports of:

>> from langchain_community.document_loaders import WikipediaLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.document_loaders import WikipediaLoader, ArxivLoader
/content/imports.py:9: LangChainDeprecationWarning: Importing ArxivLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import ArxivLoader

with new imports of:

>> from langchain_community.document_loaders import ArxivLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langcha

In [ ]:
!python test_multi_shot.py


/content/imports.py:9: LangChainDeprecationWarning: Importing WikipediaLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import WikipediaLoader

with new imports of:

>> from langchain_community.document_loaders import WikipediaLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.document_loaders import WikipediaLoader, ArxivLoader
/content/imports.py:9: LangChainDeprecationWarning: Importing ArxivLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import ArxivLoader

with new imports of:

>> from langchain_community.document_loaders import ArxivLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langcha

In [ ]:
!python test_rag.py


/content/imports.py:9: LangChainDeprecationWarning: Importing WikipediaLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import WikipediaLoader

with new imports of:

>> from langchain_community.document_loaders import WikipediaLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.document_loaders import WikipediaLoader, ArxivLoader
/content/imports.py:9: LangChainDeprecationWarning: Importing ArxivLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import ArxivLoader

with new imports of:

>> from langchain_community.document_loaders import ArxivLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langcha